![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

### <center>  Grupo 22 </center>

<center>David Alejandro Rojas Castro - da.rojasc123@uniandes.edu.co </center>
<center>Camila Malagón Suarez - c.malagons@uniandes.edu.co</center>
<center>Luis David Gutierrez - ld.gutierrezl1@uniandes.edu.co</center>
<center>David Zapata Vásquez - d.zapata11@uniandes.edu.co</center>

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación de librerías
%matplotlib inline
import pandas as pd

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,0,0,0,1,0,0,0
11,13995,2014,39972,0,0,0,0,1,0,0
167,17941,2016,18989,0,0,0,0,0,1,0
225,12493,2014,51330,0,0,0,1,0,0,0
270,7994,2007,116065,0,1,0,0,0,0,0


In [3]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [4]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [5]:
import numpy as np

# Función para calcular la media de los precios
def mean(values):
    return np.mean(values)

# Función para calcular la varianza total
def variance(values, split_value):
    return np.sum((values - split_value) ** 2)

# Función para evaluar posibles splits
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Función para calcular el mejor split
def get_split(dataset):
    b_index, b_value, b_score, b_groups = None, None, float('inf'), None
    for index in range(len(dataset[0])-1):
        unique_values = set([row[index] for row in dataset])
        for value in unique_values:
            groups = test_split(index, value, dataset)
            y_left = [row[-1] for row in groups[0]]
            y_right = [row[-1] for row in groups[1]]
            if len(y_left) == 0 or len(y_right) == 0:
                continue  # Evita splits que no dividan los datos adecuadamente
            p_left, p_right = mean(y_left), mean(y_right)
            current_score = variance(y_left, p_left) + variance(y_right, p_right)
            if current_score < b_score:
                b_index, b_value, b_score, b_groups = index, value, current_score, groups
    if b_groups is None and b_index is not None:  # Añadir un split por defecto si no se encuentra ninguno
        median = np.median([row[b_index] for row in dataset])
        b_groups = test_split(b_index, median, dataset)
    elif b_groups is None:  # Si todos los intentos fallan
        b_index = 0  # Uso el primer índice disponible como último recurso
        median = np.median([row[b_index] for row in dataset])
        b_groups = test_split(b_index, median, dataset)
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

# Función para crear una hoja terminal
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return mean(outcomes)

# Función para crear divisiones de nodos hijos o hacer terminales
def split(node, max_depth, depth):
    if 'groups' not in node or node['groups'] is None:
        node['left'] = node['right'] = to_terminal([row[-1] for row in dataset])
        return
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    node['left'] = get_split(left)
    split(node['left'], max_depth, depth+1)
    node['right'] = get_split(right)
    split(node['right'], max_depth, depth+1)

# Función para construir un árbol
def build_tree(train, max_depth):
    root = get_split(train)
    split(root, max_depth, 1)
    return root

# Función para hacer una predicción con el árbol
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Convertir datos a una lista de listas
dataset = np.column_stack((X_train, y_train)).tolist()

# Construir el árbol
tree = build_tree(dataset, max_depth=3)

# Función para calcular RMSE
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return np.sqrt(mean_error)

# Hacer predicciones en el conjunto de test
predictions = [predict(tree, row.tolist()) for row in X_test.values]
actual = y_test.tolist()

# Calcular RMSE y MAE
rmse = rmse_metric(actual, predictions)
mae = np.mean(np.abs(np.array(actual) - np.array(predictions)))

print("RMSE en el conjunto de test:", rmse)
print("MAE en el conjunto de test:", mae)

RMSE en el conjunto de test: 1935.1155006739648
MAE en el conjunto de test: 1458.0246928139518


**Análisis de resultados:**
RMSE (Root Mean Square Error): 1935.12
MAE (Mean Absolute Error): 1458.02

**Conclusión del Análisis:**
El modelo de árbol de decisión manual alcanzó un **RMSE de 1935.12** y un **MAE de 1458.02** en el conjunto de test. Estos valores proporcionan una medida de cuánto se desvían las predicciones del modelo de los valores reales. En este contexto, un RMSE de 1935.12 indica que, en promedio, las predicciones del modelo pueden estar dentro de un rango de aproximadamente 1935 unidades monetarias del precio real del automóvil. Similarmente, el MAE de 1458.02 sugiere que, en promedio, los errores absolutos de las predicciones están alrededor de 1458 unidades.

**Interpretación:**
* **Magnitud de los errores:** Los valores de RMSE y MAE son relativamente altos, lo que podría indicar que el modelo, aunque funcional, no captura completamente la complejidad o las relaciones subyacentes en los datos. Esto es común en modelos de árboles de decisión simples sin refinamiento o ajuste de parámetros, especialmente cuando se construyen manualmente sin optimizaciones avanzadas.
* **Impacto de los errores grandes:** El RMSE mayor que el MAE sugiere que hay varios errores grandes que afectan más significativamente al RMSE debido a su naturaleza de penalización cuadrática. Esto podría ser indicativo de sobreajuste en ciertas partes del árbol o de la incapacidad del modelo para manejar adecuadamente las variaciones extremas en los datos.

**Implicaciones:**
Este modelo podría servir como un punto de partida básico, pero claramente hay espacio para mejoras. El desempeño actual puede no ser suficiente para aplicaciones prácticas donde se requieren predicciones precisas de precios de automóviles, como en sistemas de recomendación para compradores de autos o en aplicaciones de inventario para vendedores de autos.

### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de regresión y comenten sobre el desempeño del modelo.

In [6]:
import numpy as np

def bootstrap_sample(data):
    n_samples = len(data)
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return data[indices]

def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return np.mean(predictions)  # Para regresión

# Crear múltiples árboles de decisión
n_trees = 10
trees = []

for _ in range(n_trees):
    sample = bootstrap_sample(np.array(dataset))
    tree = build_tree(sample.tolist(), max_depth=3)
    trees.append(tree)

# Evaluar el ensamble en el conjunto de test
predictions = [bagging_predict(trees, row.tolist()) for row in X_test.values]
actual = y_test.tolist()

# Calcular RMSE y MAE
rmse = rmse_metric(actual, predictions)
mae = np.mean(np.abs(np.array(actual) - np.array(predictions)))

print("RMSE en el conjunto de test:", rmse)
print("MAE en el conjunto de test:", mae)

RMSE en el conjunto de test: 1816.738261051669
MAE en el conjunto de test: 1353.8057201768336


**Análisis de resultados:**

Bagging Manual:

RMSE: 1816.74

MAE: 1353.81

**Observaciones:**
* **Reducción de Error:** El modelo de bagging ha reducido tanto el RMSE como el MAE en comparación con un único árbol de decisión. El RMSE se ha reducido en aproximadamente 118.38 unidades y el MAE en aproximadamente 104.22 unidades.

**Interpretación del RMSE y MAE:**
* **RMSE (Root Mean Square Error):** Este mide la magnitud de los errores entre los valores predichos y los valores reales. Dado que el RMSE penaliza más los errores grandes (por el cuadrado de las diferencias), una reducción en el RMSE sugiere una mejora en la precisión del modelo, especialmente en la reducción de grandes errores de predicción.
* **MAE (Mean Absolute Error):** Este mide el error promedio en las predicciones; es menos sensible a los valores atípicos que el RMSE. La reducción del MAE indica que, en promedio, las predicciones del modelo de bagging están más cerca de los valores reales.

**Conclusiones:**
* **Efectividad del Bagging:** La mejora en ambos, RMSE y MAE, para el modelo de bagging indica que este método puede ser más efectivo para este conjunto de datos específico. El bagging ha ayudado probablemente a reducir la varianza del modelo, haciéndolo más robusto contra el sobreajuste y mejorando la precisión general en los datos de prueba.
* **Sobreajuste:** Dado que el bagging utiliza múltiples modelos (árboles en este caso) entrenados en diferentes subconjuntos del dataset, es menos probable que el modelo resultante se sobreajuste a los datos de entrenamiento en comparación con un solo árbol de decisión. Esto podría explicar la mejora en la precisión observada con el modelo de bagging.
* **Recomendaciones:** Para este conjunto de datos y problema específico (predicción del precio de automóviles), el uso de bagging presenta una clara ventaja sobre el uso de un solo árbol de decisión. Podría ser beneficioso explorar aún más los métodos de ensamblaje, quizás aumentando el número de árboles en el bagging o utilizando otras técnicas de ensamblaje como Random Forest o Boosting para comparar rendimientos.

### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de regresión y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 3


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para regresión  y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 4


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [ ]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de regresión con la librería sklearn y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 6


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [ ]:
# Celda 7


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
# Celda 8
